In [4]:
!pip install -q gradio scikit-learn pandas


In [6]:
import pandas as pd

# Sample taxi fare dataset
data = {
    "Trip_Distance_km": [12.5, 5.0, 7.3, 15.2, 3.0, 8.6, 10.0, 4.5, 20.0, 6.7],
    "Time_of_Day": ["Morning", "Night", "Afternoon", "Evening", "Morning", "Night", "Afternoon", "Evening", "Morning", "Night"],
    "Day_of_Week": ["Monday", "Friday", "Wednesday", "Sunday", "Tuesday", "Saturday", "Thursday", "Monday", "Friday", "Wednesday"],
    "Passenger_Count": [2, 1, 3, 4, 1, 2, 1, 2, 5, 1],
    "Traffic_Conditions": ["Moderate", "Heavy", "Light", "Heavy", "Light", "Moderate", "Light", "Moderate", "Heavy", "Light"],
    "Weather": ["Clear", "Rainy", "Clear", "Stormy", "Clear", "Cloudy", "Clear", "Foggy", "Rainy", "Clear"],
    "Base_Fare": [50]*10,
    "Per_Km_Rate": [12]*10,
    "Per_Minute_Rate": [2]*10,
    "Trip_Duration_Minutes": [35, 20, 25, 50, 10, 30, 28, 18, 60, 22],
    "Trip_Price": [204, 160, 174.6, 304, 96, 202.8, 206, 142.2, 384, 164.4]
}

df = pd.DataFrame(data)
df.to_csv("taxi_data.csv", index=False)
print("✅ taxi_data.csv created")


✅ taxi_data.csv created


In [7]:
!pip install -q gradio


In [8]:
import pandas as pd
import gradio as gr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load dataset
df = pd.read_csv("taxi_data.csv")
df.dropna(inplace=True)

# Encode categorical features
label_encoders = {}
categorical_cols = df.select_dtypes(include='object').columns.tolist()

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Split features and target
X = df.drop("Trip_Price", axis=1)
y = df["Trip_Price"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)

# Prediction function
def predict_fare(*inputs):
    try:
        row = {}
        for i, col in enumerate(X.columns):
            if col in categorical_cols:
                if inputs[i] not in label_encoders[col].classes_:
                    return f"❌ Invalid input: {inputs[i]} for {col}"
                row[col] = label_encoders[col].transform([inputs[i]])[0]
            else:
                row[col] = float(inputs[i])

        input_df = pd.DataFrame([row])
        prediction = model.predict(input_df)[0]
        return f"""
## 💰 Predicted Taxi Fare: ₹{prediction:.2f}

Model R² Score: {accuracy:.2f}
"""
    except Exception as e:
        return f"❌ Error: {e}"

# Create Gradio Inputs
gr_inputs = []

for col in X.columns:
    if col in categorical_cols:
        gr_inputs.append(gr.Dropdown(choices=list(label_encoders[col].classes_), label=col.replace('_', ' ').title()))
    else:
        min_val = float(df[col].min())
        max_val = float(df[col].max())
        mean_val = float(df[col].mean())
        gr_inputs.append(gr.Slider(minimum=min_val, maximum=max_val, value=mean_val, step=1, label=col.replace('_', ' ').title()))

# Launch Gradio Interface
demo = gr.Interface(
    fn=predict_fare,
    inputs=gr_inputs,
    outputs=gr.Markdown(),
    title="🚖 Taxi Fare Prediction",
    description="Enter trip details to predict estimated fare using machine learning."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8855220ed4db42c4d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
